<a href="https://www.kaggle.com/code/nathfavour/kaggle-openai?scriptVersionId=247942549" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Environment Detection and Setup
import os
import sys
from pathlib import Path

# Detect environment
IS_KAGGLE = '/kaggle' in os.getcwd()
IS_LOCAL = not IS_KAGGLE

print(f"Python version: {sys.version}")
print(f"Environment: {'Kaggle ☁️' if IS_KAGGLE else 'Local 🏠'}")

if IS_KAGGLE:
    # Kaggle Environment Setup
    print(f"Working directory: {os.getcwd()}")
    print(f"Input data: {os.listdir('/kaggle/input') if os.path.exists('/kaggle/input') else 'No input data'}")
    print(f"GPU available: {os.environ.get('KAGGLE_GPU_TYPE', 'None')}")
    
    WORK_DIR = '/kaggle/working'
    INPUT_DIR = '/kaggle/input'
    
    # Install required packages for Kaggle
    import subprocess
    kaggle_packages = ['openai', 'python-dotenv']
    for package in kaggle_packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
            print(f"✓ Installed {package}")
        except:
            print(f"✗ Failed to install {package}")
            
else:
    # Local Environment Setup
    repo_root = Path(__file__).parent.parent if '__file__' in globals() else Path('../')
    sys.path.append(str(repo_root))
    
    WORK_DIR = str(repo_root / 'notebooks')
    INPUT_DIR = str(repo_root / 'data')
    
    print(f"Repository root: {repo_root}")
    print(f"Work directory: {WORK_DIR}")

print("✓ Environment setup complete!")

Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Environment: Kaggle ☁️
Working directory: /kaggle/working
Input data: ['openai-to-z-challenge']
GPU available: None
✓ Installed openai
✓ Installed python-dotenv
✓ Environment setup complete!


In [2]:
# Import packages and configurations
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import requests
import json

# Environment-specific imports and configurations
if IS_KAGGLE:
    # === KAGGLE CONFIGURATION ===
    print("🔧 Setting up Kaggle configuration...")
    
    # Kaggle-specific imports
    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        OPENAI_API_KEY = user_secrets.get_secret("openai_api_key")
        print("✓ OpenAI API key loaded from Kaggle secrets")
    except:
        OPENAI_API_KEY = None
        print("⚠ OpenAI API key not found in Kaggle secrets")
    
    # Inline configuration for Kaggle
    class Config:
        # API Keys
        OPENAI_API_KEY = OPENAI_API_KEY
        
        # Project settings
        PROJECT_NAME = 'openai_to_z_challenge'
        DATA_DIR = INPUT_DIR
        OUTPUT_DIR = WORK_DIR
        
        # Study area (Australia - changed from Amazon for this project)
        STUDY_BOUNDS = {
            'west': 110.0,   # Western Australia
            'south': -45.0,  # Southern Australia
            'east': 155.0,   # Eastern Australia
            'north': -10.0   # Northern Australia
        }
        
        # OpenAI model settings
        OPENAI_MODEL = 'gpt-4o-mini'  # Use available model
        MAX_TOKENS = 4000
        TEMPERATURE = 0.3
        
        # Kaggle-specific settings
        KAGGLE_WORK_DIR = WORK_DIR
        KAGGLE_INPUT_DIR = INPUT_DIR
    
    # Inline OpenAI client for Kaggle
    try:
        import openai
        
        class OpenAIAnalyzer:
            def __init__(self):
                if Config.OPENAI_API_KEY:
                    self.client = openai.OpenAI(api_key=Config.OPENAI_API_KEY)
                    print("✓ OpenAI client initialized")
                else:
                    self.client = None
                    print("⚠ OpenAI client not initialized - no API key")
            
            def test_connection(self):
                if not self.client:
                    return False
                try:
                    response = self.client.chat.completions.create(
                        model="gpt-3.5-turbo",
                        messages=[{"role": "user", "content": "Hello"}],
                        max_tokens=10
                    )
                    return True
                except:
                    return False
        
        openai_analyzer = OpenAIAnalyzer()
        
    except ImportError:
        print("⚠ OpenAI package not available")
        openai_analyzer = None

else:
    # === LOCAL CONFIGURATION ===
    print("🔧 Setting up local configuration...")
    
    # Import local modules
    try:
        from src.config import Config
        from src.ai_analysis.openai_client import OpenAIAnalyzer
        print("✓ Local modules imported successfully")
        
        # Initialize OpenAI analyzer
        openai_analyzer = OpenAIAnalyzer()
        
    except ImportError as e:
        print(f"⚠ Could not import local modules: {e}")
        print("💡 Make sure you're running from the repo root or install requirements.txt")
        
        # Fallback config for local development
        class Config:
            STUDY_BOUNDS = {
                'west': 110.0,
                'south': -45.0, 
                'east': 155.0,
                'north': -10.0
            }
            PROJECT_NAME = 'openai_to_z_challenge'
        
        openai_analyzer = None

# Try to import optional geospatial packages
try:
    import geopandas as gpd
    print("✓ GeoPandas available")
    geo_available = True
except ImportError:
    print("⚠ GeoPandas not available")
    geo_available = False

print(f"\n📊 Setup Summary:")
print(f"Environment: {'Kaggle' if IS_KAGGLE else 'Local'}")
print(f"Study area bounds: {Config.STUDY_BOUNDS}")
print(f"Geospatial support: {geo_available}")
print(f"OpenAI integration: {'✓' if openai_analyzer else '✗'}")

🔧 Setting up Kaggle configuration...
⚠ OpenAI API key not found in Kaggle secrets
⚠ OpenAI client not initialized - no API key
✓ GeoPandas available

📊 Setup Summary:
Environment: Kaggle
Study area bounds: {'west': 110.0, 'south': -45.0, 'east': 155.0, 'north': -10.0}
Geospatial support: True
OpenAI integration: ✓
